In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_selector, make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline, make_union, Pipeline, FeatureUnion

In [9]:
%%sh 
ls ../data

ILHA_SOLTEIRA.csv


In [13]:
import os
os.listdir("../data")

['ILHA_SOLTEIRA.csv']

In [32]:
data = (
    pd.read_csv(
        "../data/ILHA_SOLTEIRA.csv", 
        decimal=".", 
        delimiter=",",
    )
    .assign(
        Date=lambda df: pd.to_datetime(df.Date, errors="raise", format="%d-%m-%Y"),
    )
    .set_index("Date")
    .reindex()
)
data

,Tmean,Tmax,Tmin,RHmean,RHmax,RHmin,Pmean,Rn,Rl,G,PAR,ETcat,ET0pm,ET0cat,U2max,U2mean,U2dir,Rain,Insolation
Date,,,,,,,,,,,,,,,,,,,
1999-12-31,26.7,33.2,20.2,75.3,NaN,NaN,97.4,20.8,13.8,0.0,NaN,7.2,4.7,5.8,NaN,1.5,NaN,9.9,6.5
2000-01-01,27.1,33.7,20.5,60.8,NaN,NaN,97.4,23.4,14.8,0.0,NaN,5.8,4.9,4.1,NaN,0.9,NaN,0.3,8.0
2000-01-02,26.1,31.2,21.0,76.1,NaN,NaN,97.4,17.0,11.5,0.0,NaN,7.7,6.8,5.4,NaN,2.1,NaN,30.4,3.0
2000-01-03,23.5,28.7,18.3,71.7,NaN,NaN,97.4,14.5,9.8,0.0,NaN,5.9,6.3,4.1,NaN,2.1,NaN,24.3,1.0
2000-01-04,25.9,31.7,20.0,74.1,NaN,NaN,97.4,25.3,16.3,0.0,NaN,8.2,5.8,6.5,NaN,1.3,NaN,0.0,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-23,26.3,33.0,21.5,88.9,100.0,65.0,97.4,9.2,NaN,NaN,196.8,NaN,2.4,NaN,12.9,0.6,68.7,35.1,0.0
2025-02-24,26.2,33.0,20.9,87.9,100.0,62.8,97.6,11.4,NaN,NaN,243.8,NaN,2.7,NaN,5.8,0.4,26.8,1.5,0.0
2025-02-25,27.1,33.2,23.4,86.2,98.7,56.1,97.5,12.5,NaN,NaN,273.2,NaN,3.2,NaN,4.9,0.8,45.8,0.0,0.2


In [26]:
data.describe()

,Date,Tmean,Tmax,Tmin,RHmean,RHmax,RHmin,Pmean,Rn,Rl,G,PAR,ETcat,ET0pm,ET0cat,U2max,U2mean,U2dir,Rain,Insolation
count,9190,9151.000000,9147.000000,9149.000000,9151.000000,9090.000000,9088.000000,9118.000000,9124.000000,6394.000000,4354.000000,8978.000000,5056.000000,9160.000000,5056.000000,9088.000000,9150.000000,9058.000000,9137.000000,9053.000000
mean,2012-07-29 16:08:02.611534080,25.118905,31.885165,19.646606,70.019364,91.305589,44.897667,97.393036,17.123630,10.671270,0.258567,305.989129,6.261214,4.038570,4.277888,5.887203,1.254940,117.617874,3.540845,6.296576
min,1999-12-31 00:00:00,10.000000,11.200000,3.200000,23.100000,34.100000,7.200000,0.000000,0.000000,-87.500000,-6.200000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2006-04-15 06:00:00,23.500000,30.000000,17.600000,60.400000,87.100000,33.200000,97.200000,13.200000,7.700000,-0.100000,224.125000,4.500000,3.000000,3.100000,4.600000,0.800000,63.200000,0.000000,3.700000
50%,2012-07-29 12:00:00,25.600000,32.300000,20.600000,71.500000,94.500000,44.500000,97.400000,16.900000,10.400000,0.200000,298.750000,6.200000,4.000000,4.200000,5.600000,1.200000,96.500000,0.000000,6.500000
75%,2018-11-12 18:00:00,27.300000,34.400000,22.200000,81.000000,98.500000,55.425000,97.700000,21.000000,13.900000,0.700000,378.600000,8.100000,5.000000,5.500000,6.700000,1.600000,151.100000,0.800000,9.300000
max,2025-02-27 00:00:00,35.500000,44.600000,32.500000,100.000000,100.000000,100.000000,98.900000,35.400000,34.100000,6.200000,929.000000,26.100000,8.700000,18.000000,21.600000,3.900000,359.900000,132.300000,16.100000
std,NaN,3.282293,3.968472,3.508942,14.367337,9.507194,15.303172,2.072098,5.892244,4.474517,0.703597,121.084352,2.694203,1.391142,1.754828,1.758054,0.563957,77.575787,9.972730,3.636980


In [39]:
data.isna().sum().sort_values(ascending=False)

G             4836
ETcat         4134
ET0cat        4134
Rl            2796
PAR            212
Insolation     137
U2dir          132
U2max          102
RHmin          102
RHmax          100
Pmean           72
Rn              66
Rain            53
Tmax            43
Tmin            41
U2mean          40
Tmean           39
RHmean          39
ET0pm           30
dtype: int64